In [1]:
from dotenv import load_dotenv # This is for later use but let's not forget it

load_dotenv()  # load environment variables from .env file

from haystack import Document
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

documents = [
    Document(content="Mark lives in Berlin"),
    Document(content="Jean lives in Paris"),
    Document(content="Mark speaks Turkish"),
    Document(content="Jean was born in Belgium"),
    ]

# Add documents to the document_store
document_store = InMemoryDocumentStore()
document_store.write_documents(documents)

# Create the retriever with the document_store
retriever = InMemoryBM25Retriever(document_store)

/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(
/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack import Pipeline

# Create prompt builder component
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=prompt_template)

# Create generator component
model_name = "gpt-4o-mini"
llm = OpenAIGenerator(model=model_name)

# Build pipeline and create connections between components
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("generator", llm)

rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "generator.prompt")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [3]:
from pathlib import Path
rag_pipeline.draw(Path("pipeline.png"))

In [4]:
# Run pipeline
questions = [
    "Where does Mark live?",
    "What language does Mark speak?",
    "Where was Jean born?",
    "Where does Jean live?",
]

for query in questions:
    result = rag_pipeline.run({"retriever": {"query": query}, "prompt_builder": {"question": query}})
    llm_response = result["generator"]["replies"][0]

    print(f"User: {query}")
    print(f"Assistant: {llm_response}\n")

User: Where does Mark live?
Assistant: Mark lives in Berlin.

User: What language does Mark speak?
Assistant: Mark speaks Turkish.

User: Where was Jean born?
Assistant: Jean was born in Belgium.

User: Where does Jean live?
Assistant: Jean lives in Paris.



In [5]:
# Create a new, complete chatbot pipeline with memory
# We need to create each component again in order to be able to use them in a new pipeline

# Create the retriever with the document_store
retriever = InMemoryBM25Retriever(document_store)

# Create prompt builder component
prompt_template = """
Given these documents and the current conversation, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}

Below is the current conversation consisting of interleaving human and assistant messages.
Current Conversation:
{% for chat in chat_history %}
    {{ chat.role.name }}:  {{ chat.content }}
{% endfor %}

Answer:
"""
prompt_builder = PromptBuilder(template=prompt_template)

In [6]:
# Create generator component
model_name = "gpt-4o-mini"
llm = OpenAIGenerator(model=model_name)

# Build pipeline and create connections between components
chat_pipeline = Pipeline()
chat_pipeline.add_component("retriever", retriever)
chat_pipeline.add_component("prompt_builder", prompt_builder)
chat_pipeline.add_component("generator", llm)

chat_pipeline.connect("retriever.documents", "prompt_builder.documents")
chat_pipeline.connect("prompt_builder.prompt", "generator.prompt")


🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)

In [18]:
# Define the chat_history that will store ChatMessages.
from haystack.dataclasses.chat_message import ChatMessage
from typing import List
chat_history: List[ChatMessage] = []

In [20]:
query = "Where does Mark live?"
result = chat_pipeline.run({"retriever": {"query": query}, 
                            "prompt_builder": {"question": query, 
                                               "chat_history": chat_history}
                                               })
llm_response = result["generator"]["replies"][0]
print(f"llm_response: {llm_response}")
chat_history.append(ChatMessage.from_user(query))
chat_history.append(ChatMessage.from_assistant(llm_response))

query = "What did I ask you earlier and what was your response?"
result = chat_pipeline.run({"retriever": {"query": query}, "prompt_builder": {"question": query, "chat_history": chat_history}})
llm_response = result["generator"]["replies"][0]
print(f"llm_response: {llm_response}")

llm_response: Mark lives in Berlin.
llm_response: You asked me earlier, "Where does Mark live?" and my response was "Mark lives in Berlin."


In [22]:
print('k')

k
